In [ ]:
import pandas as pd
import json
import re
import itertools
import textdistance
import numpy as np
import random as rd
from random import seed
from random import randint
from random import choices
from sklearn.cluster import AgglomerativeClustering
import math
from collections import Counter
from matplotlib import pyplot as plt
import statistics

In [ ]:
#import data and get every tv as a single element
with open('TVs-all-merged.json') as f:
    data = json.load(f)

print(len(data.keys()))

amount_of_descriptions = 0
for key in data.keys():
    amount_of_descriptions += len(data[key])
print(amount_of_descriptions)

new_data = {}
i = 1
for key in data.keys():
    for description in data[key]:
        new_data[i] = description
        i+=1
print(len(new_data.keys()))

print(max(new_data.keys()))



In [ ]:
#create list with all brands 
list_brands=[]

for i in range(1,len(new_data.keys())+1):
    if 'Brand' in new_data[i]['featuresMap']:
        brand = new_data[i]['featuresMap']['Brand']
        list_brands = list_brands + [brand]
    elif 'Brand Name' in new_data[i]['featuresMap']:
        brand2 = new_data[i]['featuresMap']['Brand Name']
        list_brands = list_brands + [brand2]

for i in range(0,len(list_brands)):
    list_brands[i] = list_brands[i].lower()

list_brands = list(dict.fromkeys(list_brands))
list_brands

In [ ]:
def get_cosine_similarity(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
    sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator


def text_to_vector(text):
    WORD = re.compile(r"\w+")
    words = WORD.findall(text)
    return Counter(words)

In [ ]:
def create_hash(num: int):
    for i in range(0, num):
        xvar = rd.sample(range(1, num+2),num)
        yvar = rd.sample(range(0,num+2),num)
    return (xvar, yvar)


In [ ]:
def model_LSH_only(dataset, alltitles_list, modelID_list, bandlength):

    modelID = []
    for i in dataset:
        modelID.append(modelID_list[i])

    titles_dataset = []
    for i in dataset:
        titles_dataset.append(alltitles_list[i])
    
    complete_list_words = []
    for i in range(0, len(titles_dataset)):
        list_title_words = str.split(titles_dataset[i])
        complete_list_words += list_title_words

    for i in range(0,len(complete_list_words)):
        complete_list_words[i] = complete_list_words[i].lower()    

    pattern = re.compile("\d+")

    for i in range(0,len(complete_list_words)-1):
        if (complete_list_words[i+1] == 'inch' or complete_list_words[i+1] == '”' or complete_list_words[i+1] == '"' or complete_list_words[i+1] == '-inch') and bool(pattern.match(complete_list_words[i])):
            new_word = complete_list_words[i] + 'inch'
            complete_list_words[i] = new_word 
        elif (complete_list_words[i+1] == 'hz' or complete_list_words[i+1] == 'hertz' or complete_list_words[i+1] == '-hz') and bool(pattern.match(complete_list_words[i])):
            new_word2= complete_list_words[i] + 'hz'
            complete_list_words[i] = new_word2
    
    new_list1 = [s.replace("(", "") for s in complete_list_words]
    new_list2 = [s.replace("[", "") for s in new_list1]
    r = re.compile("philips|samsung|supersonic|sharp|nec|toshiba|\
    hisense|sony|lg|sanyo|coby|panasonic|sansui|vizio|viewsonic|sunbritetv|haier|\
    proscan|jvc|pyle|lg electronics|sceptre|magnavox|mitsubishi|compaq|hannspree|\
    sceptre inc.|upstar|seiki|rca|craig|affinity|jvc tv|naxa|westinghouse|epson|hp|\
    elo|pansonic|hello kitty|sigmac|[a-z0-9]*[0-9]+[^0-9,(\s]*|[^0-9,(\s]*[0-9]+[a-z0-9]*")
    filtered_list = list(filter(r.match, new_list2))

    filtered_list_final = [re.sub(r'\"|\”|\'', 'inch', item) for item in filtered_list]
    filtered_list_final2 = [re.sub(r'-inch', 'inch', item) for item in filtered_list_final]
    filtered_list_final3 = [re.sub(r'-hz|hertz', 'hz', item) for item in filtered_list_final2]
    filtered_list_final4 = [s.replace(".0", "") for s in filtered_list_final3]
    filtered_list_final5 = [s.replace(")", "") for s in filtered_list_final4]

    #create list of words per tv
    list_per_tv = []
    for i in dataset:
        list_per_tv.append(alltitles_list[i].split())

    for i in range(0,len(list_per_tv)):
        for j in range(0, len(list_per_tv[i])):
            list_per_tv[i][j] = list_per_tv[i][j].lower()    

    #filter words we are interested in
    pattern = re.compile("\d+")
                
    for i in range(0,len(list_per_tv)):
        for j in range(0, len(list_per_tv[i])-1):
            if (list_per_tv[i][j+1] == 'inch' or list_per_tv[i][j+1] == '”' or list_per_tv[i][j+1] == '"' or list_per_tv[i][j+1] == '-inch') and bool(pattern.match(list_per_tv[i][j])):
                new_word3 = list_per_tv[i][j] + 'inch'
                list_per_tv[i][j] = new_word3
            elif (list_per_tv[i][j+1] == 'hz' or list_per_tv[i][j+1] == 'hertz' or list_per_tv[i][j+1] == '-hz') and bool(pattern.match(list_per_tv[i][j])):
                new_word4 = list_per_tv[i][j] + 'hz'
                list_per_tv[i][j] = new_word4

    filtered_per_tv = []            
    for i in range(0,len(list_per_tv)): 
        new_list3 = [s.replace("(", "") for s in list_per_tv[i]]
        new_list4 = [s.replace("[", "") for s in new_list3]
        r = re.compile("philips|samsung|supersonic|sharp|nec|toshiba|\
        hisense|sony|lg|sanyo|coby|panasonic|sansui|vizio|viewsonic|sunbritetv|haier|\
        proscan|jvc|pyle|lg electronics|sceptre|magnavox|mitsubishi|compaq|hannspree|\
        sceptre inc.|upstar|seiki|rca|craig|affinity|jvc tv|naxa|westinghouse|epson|hp|\
        elo|pansonic|hello kitty|sigmac|[a-z0-9]*[0-9]+[^0-9,(\s]*|[^0-9,(\s]*[0-9]+[a-z0-9]*")
        new_list5 = list(filter(r.match, new_list4))
        filtered_per_tv.append(new_list5)

    #clean the words per tv further
    filtered_per_tv_final = []    
    for i in range(0,len(filtered_per_tv)):
        new_list6 = [re.sub(r'\"|\”|\'', 'inch', item) for item in filtered_per_tv[i]]
        new_list7 = [re.sub(r'-inch', 'inch', item) for item in new_list6]
        new_list8 = [re.sub(r'-hz|hertz', 'hz', item) for item in new_list7]
        new_list9 = [s.replace(".0", "") for s in new_list8]
        new_list10 = [s.replace(")", "") for s in new_list9]
    
        filtered_per_tv_final.append(new_list10)    
        
        
    unique_filtered_list = list(dict.fromkeys(filtered_list_final5))

    #create zero dataframe
    s = (len(unique_filtered_list),len(modelID))
    A = np.zeros(s)
    df = pd.DataFrame(A, index=unique_filtered_list, columns=modelID)
    
    #create binary matrix with all model words present for each tv
    for z in range(0, len(modelID)):
        for j in range(0, len(filtered_per_tv_final[z])):
            for i in range(0, len(unique_filtered_list)):
                if filtered_per_tv_final[z][j] == unique_filtered_list[i]:
                    df.iloc[i,z]= 1

    seed(0)
    xvar = []
    yvar =[]
    num_of_hashes = int(len(unique_filtered_list)/2)-int(len(unique_filtered_list)/2)%bandlength
    primes = []
    for number in range(num_of_hashes,1000):  
        if number > 1:  
            for i in range(2,number):  
                if (number % i) == 0:  
                    break  
            else:  
                primes.append(number)
       
    randomprime = rd.sample(primes, 1)
    mod = randomprime[0]
    variables = create_hash(num_of_hashes)
    hashvalue = [ [ 0 for i in range(len(df)) ] for j in range(num_of_hashes) ]
    for j in range(1, num_of_hashes+1):
        for i in range(1,len(df)):
            hashvalue[j-1][i-1] = ((variables[0][j-1]*i+variables[1][j-1])%mod)
    final = hashvalue

    transposedf = df.transpose()

    sig = [float('inf')]*num_of_hashes
    signatures = [sig]*(len(transposedf))
    signaturesdf = pd.DataFrame(signatures, index = [modelID])
    signaturesdf.columns = np.arange(1,num_of_hashes+1)

    for i in range(0,len(transposedf)): 
        for j in range(0, len(transposedf.iloc[0])): 
            if transposedf.iloc[i,j] == 1: 
                for z in range(0, len(final)):
                    if final[z][j] < signaturesdf.iloc[i,z]: 
                        signaturesdf.iloc[i,z] = final[z][j]                    

    signaturesdft = signaturesdf.transpose()

    amountofbands = num_of_hashes/bandlength
    signaturesperband = [];
    for i in range(1, int(amountofbands)+1):
            if i == 1:
                signaturesperband.append(signaturesdft.iloc[0:bandlength].astype(int))
            else:
                signaturesperband.append(signaturesdft.iloc[bandlength*(i-1):bandlength*(i)].astype(int))

    matrixwithkeyssize = (len(signaturesperband),len(modelID))
    matrixwithkeys = np.zeros(matrixwithkeyssize)
    allkeys = pd.DataFrame(matrixwithkeys, columns=modelID)

    string = ''
    for b in range(0, len(signaturesperband)):
        for j in range(0, len(signaturesperband[b].iloc[0])):
            for i in range(0, len(signaturesperband[0])):
                string = string + str(signaturesperband[b].iloc[i,j])
            allkeys.iloc[b,j] = string
            string = ''

    tvlist = ["tv" + str(i) for i in dataset]

    sizekeys = (len(allkeys.iloc[0]),len(allkeys))
    sizething = np.zeros(sizekeys)
    allkeyspertv = pd.DataFrame(sizething, index=modelID)

    tvlength = (len(tvlist),len(tvlist))
    Atvlength = np.zeros(tvlength)
    dftvlist = pd.DataFrame(Atvlength, index=tvlist, columns=tvlist)
    dftvlistint = dftvlist.astype(int)

    for i in range(0, len(allkeys)):
        allkeysperband = pd.DataFrame(allkeys.iloc[i,:])
        allkeysperband.reset_index(drop=True)
        allkeysperband.index = tvlist
        sortedkeypertv = allkeysperband.sort_values(i)

        masterlist = [];
        smallerlist = [];
        for j in range(0, len(sortedkeypertv)-1):
            if j == (len(sortedkeypertv)-2):
                smallerlist.append(str(sortedkeypertv.index[j]))
                smallerlist.append(str(sortedkeypertv.index[j+1]))
                masterlist.append(smallerlist)
            elif sortedkeypertv[i][j+1] == sortedkeypertv[i][j]:
                smallerlist.append(str(sortedkeypertv.index[j]))
            else:
                smallerlist.append(str(sortedkeypertv.index[j]))
                masterlist.append(smallerlist)
                smallerlist = []

        listofcombos = []
        for z in range(0, len(masterlist)):
            for subset in itertools.combinations(masterlist[z], 2):
                listofcombos.append(subset)

        for p in range(0,len(listofcombos)):
            dftvlistint[listofcombos[p][0]][listofcombos[p][1]] = 1
            dftvlistint[listofcombos[p][1]][listofcombos[p][0]] = 1

    ## Nu heb je LSH afgerond

    arrdata = dftvlistint.to_numpy()
    newarrdata = np.triu(arrdata)
    dfnew = pd.DataFrame(newarrdata)
    
    # numofcomp2 = dftvlistint
    #All comparisons made from candidate pairs
    numofcomp2 = dfnew

    numofcomp2 = numofcomp2.replace(float('-inf'),0)
    numberofcomparisons2 = sum(numofcomp2[numofcomp2 > 0].count())
    
    ## All comparisons possible
    onematrixl = (len(dftvlistint), len(dftvlistint))
    onematrix = np.ones(onematrixl)
    onematrixup = np.triu(onematrix)
    np.fill_diagonal(onematrixup, 0)
    onematrixupdf = pd.DataFrame(onematrixup)
    numofonesindf = sum(onematrixupdf[onematrixupdf > 0].count())
    
    fracofcomparisons = numberofcomparisons2/numofonesindf

    filtered_final = []
    regex = re.compile(r'.{3,}inch')
    for i in range(0, len(filtered_per_tv_final)):
        filtered = [g for g in filtered_per_tv_final[i] if not regex.match(g)]
        filtered_final.append(filtered)

    shops = []
    for i in dataset:
         shops.append(new_data[i+1]["shop"])
    
    brands = []
    for i in dataset:
        if ('Brand' in new_data[i+1]['featuresMap']):
            brands.append(new_data[i+1]['featuresMap']['Brand'])
        elif ('Brand Name' in new_data[i+1]['featuresMap']):
            brands.append(new_data[i+1]['featuresMap']['Brand Name'])
        else:
            brands.append('no result')
    for i in range(0,len(brands)):
        brands[i] = brands[i].lower() 
    brandslist = [re.sub(r'lg electronics', 'lg', item) for item in brands]
    brandslist2 = [re.sub(r'sceptre inc.', 'sceptre', item) for item in brandslist]
    brands_final = [re.sub(r'jvc tv', 'jvc', item) for item in brandslist2]

    tvlength = (len(tvlist),len(tvlist))
    Atvlength = np.zeros(tvlength)
    dftvlist = pd.DataFrame(Atvlength, index=tvlist, columns=tvlist )
    dftvlist

    for i in range(0, len(dataset)):
        for j in range(0, len(dataset)):
            if j > i:
                if dfnew.iloc[i,j] == 1:
                    dftvlist.iloc[i,j] = get_cosine_similarity(text_to_vector(titles_dataset[i]), text_to_vector(titles_dataset[j]))
    
    for i in range(0, len(dftvlist)):
        for j in range(0, len(dftvlist)):
            if j > i and shops[i] == shops[j]:
                dftvlist.iloc[i,j] = -float('inf')
            if j > i and brands_final[i] != brands_final[j] and brands_final[i] != 'no result' and brands_final[j] != 'no result':
                dftvlist.iloc[i,j] = -float('inf')
                
    ## All comparisons, with shops and brands taken into account with LSH
    numofcomp = dftvlist
    numofcomp = numofcomp.replace(float('-inf'),0)
    numberofcomparisons = sum(numofcomp[numofcomp > 0].count())
    
    ## All possible comparisons, with shops and brands taken into account
    onematrix2 = (len(dftvlistint), len(dftvlistint))
    onematrix3 = np.ones(onematrix2)
    onematrixup2 = np.triu(onematrix3)
    np.fill_diagonal(onematrixup2, 0)
    onematrixupdf2 = pd.DataFrame(onematrixup2)
    
    for i in range(0, len(onematrixupdf2)):
        for j in range(0, len(onematrixupdf2)):
            if j > i and shops[i] == shops[j]:
                onematrixupdf2.iloc[i,j] = 0
            if j > i and brands_final[i] != brands_final[j] and brands_final[i] != 'no result' and brands_final[j] != 'no result':
                onematrixupdf2.iloc[i,j] = 0
                
    numofonesindf4 = sum(onematrixupdf[onematrixupdf2 > 0].count())
    
    fracofcomparisons2 = numberofcomparisons/numofonesindf4
    
    originalm = (len(modelID),len(modelID))
    originalA = np.zeros(originalm)
    originalcorrect = pd.DataFrame(originalA, index=modelID, columns=modelID)
    for i in range(0, len(modelID)):
        for j in range(0, len(modelID)):
            if modelID[i] == modelID[j]:
                originalcorrect.iloc[i,j] = 1
    originalcorrect.values[[np.arange(originalcorrect.shape[0])]*2] = 0
    
    falsenegatives_lsh = 0
    falsepositives_lsh = 0
    truenegatives_lsh = 0
    truepositives_lsh = 0

    for i in range(0, len(originalcorrect)):
        for j in range(0, len(originalcorrect)):
            if j > i:
                if  dfnew.iloc[i,j] == 0 and originalcorrect.iloc[i,j] == 0:
                    truenegatives_lsh = truenegatives_lsh + 1
                elif  dfnew.iloc[i,j] == 0 and originalcorrect.iloc[i,j] == 1:
                    falsenegatives_lsh = falsenegatives_lsh + 1
                elif  dfnew.iloc[i,j] == 1 and originalcorrect.iloc[i,j] == 0:
                    falsepositives_lsh = falsepositives_lsh + 1
                elif  dfnew.iloc[i,j] == 1 and originalcorrect.iloc[i,j] == 1:
                    truepositives_lsh = truepositives_lsh + 1

    f1measure_lsh = (truepositives_lsh/(truepositives_lsh+0.5*(falsenegatives_lsh+falsepositives_lsh)))

    pairquality_lsh = (truepositives_lsh)/(numberofcomparisons) 
    paircompleteness_lsh = (truepositives_lsh)/(truepositives_lsh + falsenegatives_lsh)
  
         #print("Pair quality = ", str(pairquality))
#     print("Pair completeness = ", str(paircompleteness))
    f1starmeasure_lsh = (2*pairquality_lsh*paircompleteness_lsh)/(pairquality_lsh+paircompleteness_lsh)
#     print("f1starmeasure = " + str(f1starmeasure))
    
    return (fracofcomparisons, fracofcomparisons2, f1starmeasure_lsh, f1measure_lsh, truepositives_lsh, falsepositives_lsh, falsenegatives_lsh, truenegatives_lsh, pairquality_lsh, paircompleteness_lsh)

In [ ]:
def model_LSH(dataset, alltitles_list, modelID_list, bandlength, cos_threshold, beta, delta, clustering_threshold):

    modelID = []
    for i in dataset:
        modelID.append(modelID_list[i])

    titles_dataset = []
    for i in dataset:
        titles_dataset.append(alltitles_list[i])
    
    complete_list_words = []
    for i in range(0, len(titles_dataset)):
        list_title_words = str.split(titles_dataset[i])
        complete_list_words += list_title_words

    for i in range(0,len(complete_list_words)):
        complete_list_words[i] = complete_list_words[i].lower()    

    pattern = re.compile("\d+")

    for i in range(0,len(complete_list_words)-1):
        if (complete_list_words[i+1] == 'inch' or complete_list_words[i+1] == '”' or complete_list_words[i+1] == '"' or complete_list_words[i+1] == '-inch') and bool(pattern.match(complete_list_words[i])):
            new_word = complete_list_words[i] + 'inch'
            complete_list_words[i] = new_word 
        elif (complete_list_words[i+1] == 'hz' or complete_list_words[i+1] == 'hertz' or complete_list_words[i+1] == '-hz') and bool(pattern.match(complete_list_words[i])):
            new_word2= complete_list_words[i] + 'hz'
            complete_list_words[i] = new_word2
    
    new_list1 = [s.replace("(", "") for s in complete_list_words]
    new_list2 = [s.replace("[", "") for s in new_list1]
    r = re.compile("philips|samsung|supersonic|sharp|nec|toshiba|\
    hisense|sony|lg|sanyo|coby|panasonic|sansui|vizio|viewsonic|sunbritetv|haier|\
    proscan|jvc|pyle|lg electronics|sceptre|magnavox|mitsubishi|compaq|hannspree|\
    sceptre inc.|upstar|seiki|rca|craig|affinity|jvc tv|naxa|westinghouse|epson|hp|\
    elo|pansonic|hello kitty|sigmac|[a-z0-9]*[0-9]+[^0-9,(\s]*|[^0-9,(\s]*[0-9]+[a-z0-9]*")
    filtered_list = list(filter(r.match, new_list2))

    filtered_list_final = [re.sub(r'\"|\”|\'', 'inch', item) for item in filtered_list]
    filtered_list_final2 = [re.sub(r'-inch', 'inch', item) for item in filtered_list_final]
    filtered_list_final3 = [re.sub(r'-hz|hertz', 'hz', item) for item in filtered_list_final2]
    filtered_list_final4 = [s.replace(".0", "") for s in filtered_list_final3]
    filtered_list_final5 = [s.replace(")", "") for s in filtered_list_final4]

    #create list of words per tv
    list_per_tv = []
    for i in dataset:
        list_per_tv.append(alltitles_list[i].split())

    for i in range(0,len(list_per_tv)):
        for j in range(0, len(list_per_tv[i])):
            list_per_tv[i][j] = list_per_tv[i][j].lower()    

    #filter words we are interested in
    pattern = re.compile("\d+")
                
    for i in range(0,len(list_per_tv)):
        for j in range(0, len(list_per_tv[i])-1):
            if (list_per_tv[i][j+1] == 'inch' or list_per_tv[i][j+1] == '”' or list_per_tv[i][j+1] == '"' or list_per_tv[i][j+1] == '-inch') and bool(pattern.match(list_per_tv[i][j])):
                new_word3 = list_per_tv[i][j] + 'inch'
                list_per_tv[i][j] = new_word3
            elif (list_per_tv[i][j+1] == 'hz' or list_per_tv[i][j+1] == 'hertz' or list_per_tv[i][j+1] == '-hz') and bool(pattern.match(list_per_tv[i][j])):
                new_word4 = list_per_tv[i][j] + 'hz'
                list_per_tv[i][j] = new_word4

    filtered_per_tv = []            
    for i in range(0,len(list_per_tv)): 
        new_list3 = [s.replace("(", "") for s in list_per_tv[i]]
        new_list4 = [s.replace("[", "") for s in new_list3]
        r = re.compile("philips|samsung|supersonic|sharp|nec|toshiba|\
        hisense|sony|lg|sanyo|coby|panasonic|sansui|vizio|viewsonic|sunbritetv|haier|\
        proscan|jvc|pyle|lg electronics|sceptre|magnavox|mitsubishi|compaq|hannspree|\
        sceptre inc.|upstar|seiki|rca|craig|affinity|jvc tv|naxa|westinghouse|epson|hp|\
        elo|pansonic|hello kitty|sigmac|[a-z0-9]*[0-9]+[^0-9,(\s]*|[^0-9,(\s]*[0-9]+[a-z0-9]*")
        new_list5 = list(filter(r.match, new_list4))
        filtered_per_tv.append(new_list5)

    #clean the words per tv further
    filtered_per_tv_final = []    
    for i in range(0,len(filtered_per_tv)):
        new_list6 = [re.sub(r'\"|\”|\'', 'inch', item) for item in filtered_per_tv[i]]
        new_list7 = [re.sub(r'-inch', 'inch', item) for item in new_list6]
        new_list8 = [re.sub(r'-hz|hertz', 'hz', item) for item in new_list7]
        new_list9 = [s.replace(".0", "") for s in new_list8]
        new_list10 = [s.replace(")", "") for s in new_list9]
    
        filtered_per_tv_final.append(new_list10)    
        
        
    unique_filtered_list = list(dict.fromkeys(filtered_list_final5))

    #create zero dataframe
    s = (len(unique_filtered_list),len(modelID))
    A = np.zeros(s)
    df = pd.DataFrame(A, index=unique_filtered_list, columns=modelID)
    
    #create binary matrix with all model words present for each tv
    for z in range(0, len(modelID)):
        for j in range(0, len(filtered_per_tv_final[z])):
            for i in range(0, len(unique_filtered_list)):
                if filtered_per_tv_final[z][j] == unique_filtered_list[i]:
                    df.iloc[i,z]= 1

    seed(0)
    xvar = []
    yvar =[]
    num_of_hashes = int(len(unique_filtered_list)/2)-int(len(unique_filtered_list)/2)%bandlength
    primes = []
    for number in range(num_of_hashes,1000):  
        if number > 1:  
            for i in range(2,number):  
                if (number % i) == 0:  
                    break  
            else:  
                primes.append(number)
       
    randomprime = rd.sample(primes, 1)
    mod = randomprime[0]
    variables = create_hash(num_of_hashes)
    hashvalue = [ [ 0 for i in range(len(df)) ] for j in range(num_of_hashes) ]
    for j in range(1, num_of_hashes+1):
        for i in range(1,len(df)):
            hashvalue[j-1][i-1] = ((variables[0][j-1]*i+variables[1][j-1])%mod)
    final = hashvalue

    transposedf = df.transpose()

    sig = [float('inf')]*num_of_hashes
    signatures = [sig]*(len(transposedf))
    signaturesdf = pd.DataFrame(signatures, index = [modelID])
    signaturesdf.columns = np.arange(1,num_of_hashes+1)

    for i in range(0,len(transposedf)): #
        for j in range(0, len(transposedf.iloc[0])): 
            if transposedf.iloc[i,j] == 1: 
                for z in range(0, len(final)): 
                    if final[z][j] < signaturesdf.iloc[i,z]:
                        signaturesdf.iloc[i,z] = final[z][j]                    

    signaturesdft = signaturesdf.transpose()

    amountofbands = num_of_hashes/bandlength
    signaturesperband = [];
    for i in range(1, int(amountofbands)+1):
            if i == 1:
                signaturesperband.append(signaturesdft.iloc[0:bandlength].astype(int))
            else:
                signaturesperband.append(signaturesdft.iloc[bandlength*(i-1):bandlength*(i)].astype(int))

    matrixwithkeyssize = (len(signaturesperband),len(modelID))
    matrixwithkeys = np.zeros(matrixwithkeyssize)
    allkeys = pd.DataFrame(matrixwithkeys, columns=modelID)

    string = ''
    for b in range(0, len(signaturesperband)):
        for j in range(0, len(signaturesperband[b].iloc[0])):
            for i in range(0, len(signaturesperband[0])):
                string = string + str(signaturesperband[b].iloc[i,j])
            allkeys.iloc[b,j] = string
            string = ''

    tvlist = ["tv" + str(i) for i in dataset]

    sizekeys = (len(allkeys.iloc[0]),len(allkeys))
    sizething = np.zeros(sizekeys)
    allkeyspertv = pd.DataFrame(sizething, index=modelID)

    tvlength = (len(tvlist),len(tvlist))
    Atvlength = np.zeros(tvlength)
    dftvlist = pd.DataFrame(Atvlength, index=tvlist, columns=tvlist)
    dftvlistint = dftvlist.astype(int)

    for i in range(0, len(allkeys)):
        allkeysperband = pd.DataFrame(allkeys.iloc[i,:])
        allkeysperband.reset_index(drop=True)
        allkeysperband.index = tvlist
        sortedkeypertv = allkeysperband.sort_values(i)

        masterlist = [];
        smallerlist = [];
        for j in range(0, len(sortedkeypertv)-1):
            if j == (len(sortedkeypertv)-2):
                smallerlist.append(str(sortedkeypertv.index[j]))
                smallerlist.append(str(sortedkeypertv.index[j+1]))
                masterlist.append(smallerlist)
            elif sortedkeypertv[i][j+1] == sortedkeypertv[i][j]:
                smallerlist.append(str(sortedkeypertv.index[j]))
            else:
                smallerlist.append(str(sortedkeypertv.index[j]))
                masterlist.append(smallerlist)
                smallerlist = []

        listofcombos = []
        for z in range(0, len(masterlist)):
            for subset in itertools.combinations(masterlist[z], 2):
                listofcombos.append(subset)

        for p in range(0,len(listofcombos)):
            dftvlistint[listofcombos[p][0]][listofcombos[p][1]] = 1
            dftvlistint[listofcombos[p][1]][listofcombos[p][0]] = 1

    #LSH finished

    arrdata = dftvlistint.to_numpy()
    newarrdata = np.triu(arrdata)
    dfnew = pd.DataFrame(newarrdata)
    

    numofcomp2 = dfnew

    numofcomp2 = numofcomp2.replace(float('-inf'),0)
    numberofcomparisons2 = sum(numofcomp2[numofcomp2 > 0].count())
    
    ## All comparisons possible
    onematrixl = (len(dftvlistint), len(dftvlistint))
    onematrix = np.ones(onematrixl)
    onematrixup = np.triu(onematrix)
    np.fill_diagonal(onematrixup, 0)
    onematrixupdf = pd.DataFrame(onematrixup)
    numofonesindf = sum(onematrixupdf[onematrixupdf > 0].count())
    
    fracofcomparisons = numberofcomparisons2/numofonesindf

    filtered_final = []
    regex = re.compile(r'.{3,}inch')
    for i in range(0, len(filtered_per_tv_final)):
        filtered = [g for g in filtered_per_tv_final[i] if not regex.match(g)]
        filtered_final.append(filtered)

    shops = []
    for i in dataset:
         shops.append(new_data[i+1]["shop"])
    
    brands = []
    for i in dataset:
        if ('Brand' in new_data[i+1]['featuresMap']):
            brands.append(new_data[i+1]['featuresMap']['Brand'])
        elif ('Brand Name' in new_data[i+1]['featuresMap']):
            brands.append(new_data[i+1]['featuresMap']['Brand Name'])
        else:
            brands.append('no result')
    for i in range(0,len(brands)):
        brands[i] = brands[i].lower() 
    brandslist = [re.sub(r'lg electronics', 'lg', item) for item in brands]
    brandslist2 = [re.sub(r'sceptre inc.', 'sceptre', item) for item in brandslist]
    brands_final = [re.sub(r'jvc tv', 'jvc', item) for item in brandslist2]

    tvlength = (len(tvlist),len(tvlist))
    Atvlength = np.zeros(tvlength)
    dftvlist = pd.DataFrame(Atvlength, index=tvlist, columns=tvlist )
    dftvlist

    for i in range(0, len(dataset)):
        for j in range(0, len(dataset)):
            if j > i:
                if dfnew.iloc[i,j] == 1:
                    dftvlist.iloc[i,j] = get_cosine_similarity(text_to_vector(titles_dataset[i]), text_to_vector(titles_dataset[j]))
    
    for i in range(0, len(dftvlist)):
        for j in range(0, len(dftvlist)):
            if j > i and shops[i] == shops[j]:
                dftvlist.iloc[i,j] = -float('inf')
            if j > i and brands_final[i] != brands_final[j] and brands_final[i] != 'no result' and brands_final[j] != 'no result':
                dftvlist.iloc[i,j] = -float('inf')
                
    ## All comparisons, with shops and brands taken into account with LSH
    numofcomp = dftvlist
    numofcomp = numofcomp.replace(float('-inf'),0)
    numberofcomparisons = sum(numofcomp[numofcomp > 0].count())
    
    ## All possible comparisons, with shops and brands taken into account
    onematrix2 = (len(dftvlistint), len(dftvlistint))
    onematrix3 = np.ones(onematrix2)
    onematrixup2 = np.triu(onematrix3)
    np.fill_diagonal(onematrixup2, 0)
    onematrixupdf2 = pd.DataFrame(onematrixup2)
    
    for i in range(0, len(onematrixupdf2)):
        for j in range(0, len(onematrixupdf2)):
            if j > i and shops[i] == shops[j]:
                onematrixupdf2.iloc[i,j] = 0
            if j > i and brands_final[i] != brands_final[j] and brands_final[i] != 'no result' and brands_final[j] != 'no result':
                onematrixupdf2.iloc[i,j] = 0
                
    numofonesindf4 = sum(onematrixupdf[onematrixupdf2 > 0].count())
    
    fracofcomparisons2 = numberofcomparisons/numofonesindf4
    
    
    tvlength = (len(tvlist),len(tvlist))
    Atvlength = np.zeros(tvlength)
    dfsimlev = pd.DataFrame(Atvlength)

    tvlength1 = (len(tvlist),len(tvlist))
    Atvlength1 = np.zeros(tvlength1)
    dfsimlevMW = pd.DataFrame(Atvlength1)

    list1=[]
    regex = re.compile(r'[0-9]*')
    for i in range(0, len(filtered_final)):
        list2 = [re.sub(r'[0-9]*', '', item) for item in filtered_final[i]]
        list1.append(list2)

    list3=[]
    regex = re.compile(r'[^0-9]*')
    for i in range(0, len(filtered_final)):
        list4 = [re.sub(r'[^0-9]*', '', item) for item in filtered_final[i]]
        list3.append(list4)

    #TMWM
    sum_lev_MW = 0
    sum_lev = 0
    sum_length = 0
    sum_length_allpairs = 0
    for i in range(0, len(dftvlist)):
        for j in range(0, len(dftvlist)):
            if j > i and dfnew.iloc[i,j] == 1 and dftvlist.iloc[i,j] < cos_threshold and dftvlist.iloc[i,j] >= 0:
                for z in range(0, len(filtered_final[i])):
                    for x in range(0, len(filtered_final[j])):
                        first_model = list1[i][z]
                        second_model = list1[j][x]
                        text_distance = textdistance.levenshtein.normalized_distance(first_model, second_model)
                        first_item = filtered_final[i][z]
                        second_item = filtered_final[j][x]
                        total_distance = textdistance.levenshtein.normalized_distance(first_item, second_item)
                        length_allpairs = len(filtered_final[i][z])+len(filtered_final[j][x])
                        sum_lev = sum_lev + (1-total_distance) * length_allpairs
                        sum_length_allpairs = sum_length_allpairs + length_allpairs
                        if text_distance < 0.1:
                            first_word = list3[i][z]
                            second_word = list3[j][x]
                            numeric_distance = textdistance.levenshtein.normalized_distance(first_word, second_word)
                            if numeric_distance > 0:
                                dftvlist.iloc[i,j] = -float('inf')
                                length = len(filtered_final[i][z])+len(filtered_final[j][x])
                                sum_length = sum_length + length
                            else:
                                first_string = filtered_final[i][z]
                                second_string = filtered_final[j][x]
                                string_distance = textdistance.levenshtein.normalized_distance(first_string, second_string)
                                length = len(filtered_final[i][z])+len(filtered_final[j][x])
                                sum_lev_MW = sum_lev_MW + (1-string_distance) * length
                                sum_length = sum_length + length
            if dftvlist.iloc[i,j] != 0 and sum_length != 0:
                avg_lev = sum_lev / sum_length_allpairs
                avg_lev_MW = sum_lev_MW / sum_length
                dfsimlev.iloc[i,j] = beta * dftvlist.iloc[i,j] + (1-beta) * avg_lev
                dfsimlevMW.iloc[i,j] = delta * avg_lev_MW + (1-delta) * dfsimlev.iloc[i,j]
                sum_lev_MW = 0
                sum_length = 0
                sum_lev = 0
                sum_length_allpairs = 0

    for i in range(0, len(dfsimlevMW)):
        for j in range(0, len(dfsimlevMW)):
            if j > i and shops[i] == shops[j]:
                dfsimlevMW.iloc[i,j] = -float('inf')
            if j > i and brands_final[i] != brands_final[j] and brands_final[i] != 'no result' and brands_final[j] != 'no result':
                dfsimlevMW.iloc[i,j] = -float('inf')
   
    
    finalm = (len(dfsimlevMW),len(dfsimlevMW))
    finalA = np.ones(finalm)
    onematrix = pd.DataFrame(finalA)
    dfsimlevMWone = onematrix.sub(dfsimlevMW)
    finaldissimilaritymatrix = dfsimlevMWone.replace(1, 1000)
    finaldissimilaritymatrix2 = finaldissimilaritymatrix.replace(float('inf'), 1000)
    finaldissimilaritymatrix2numpy = finaldissimilaritymatrix2.to_numpy()
    i_lower = np.tril_indices(len(tvlist), -1)
    finaldissimilaritymatrix2numpy[i_lower] = finaldissimilaritymatrix2numpy.T[i_lower]
    dffinaldissim = pd.DataFrame(finaldissimilaritymatrix2numpy)

    clustering = AgglomerativeClustering(n_clusters = None, affinity='precomputed', linkage='complete', distance_threshold=clustering_threshold, compute_full_tree= True).fit(dffinaldissim)
    labelsclust = clustering.labels_
    
    originalm = (len(modelID),len(modelID))
    originalA = np.zeros(originalm)
    originalcorrect = pd.DataFrame(originalA, index=modelID, columns=modelID)
    for i in range(0, len(modelID)):
        for j in range(0, len(modelID)):
            if modelID[i] == modelID[j]:
                originalcorrect.iloc[i,j] = 1
    originalcorrect.values[[np.arange(originalcorrect.shape[0])]*2] = 0

    clusterdm = (len(modelID),len(modelID))
    clusteredA = np.zeros(clusterdm)
    clusteredmat = pd.DataFrame(clusteredA, index=modelID, columns=modelID)
    check = pd.DataFrame(list(zip(labelsclust, modelID)))
    for i in range(0, len(modelID)):
        for j in range(0, len(modelID)):
            if check[0][i] == check[0][j]:
                clusteredmat.iloc[i,j] = 1
    clusteredmat.values[[np.arange(clusteredmat.shape[0])]*2] = 0
    #calculate performance measures
    falsenegatives = 0
    falsepositives = 0
    truenegatives = 0
    truepositives = 0
    
    for i in range(0, len(originalcorrect)):
        for j in range(0, len(originalcorrect)):
            if j > i:
                if  clusteredmat.iloc[i,j] == 0 and originalcorrect.iloc[i,j] == 0:
                    truenegatives = truenegatives + 1
                elif  clusteredmat.iloc[i,j] == 0 and originalcorrect.iloc[i,j] == 1:
                    falsenegatives = falsenegatives + 1
                elif  clusteredmat.iloc[i,j] == 1 and originalcorrect.iloc[i,j] == 0:
                    falsepositives = falsepositives + 1
                elif  clusteredmat.iloc[i,j] == 1 and originalcorrect.iloc[i,j] == 1:
                    truepositives = truepositives + 1

#     print("truepositives = "+ str(truepositives))
#     print("truenegatives = " + str(truenegatives))
#     print("falsepositives = " + str(falsepositives))
#     print("falsenegatives = "+ str(falsenegatives))
    f1measure = (truepositives/(truepositives+0.5*(falsenegatives+falsepositives)))
#     print("f1measure = " + str(f1measure))

    pairquality = (truepositives)/(numberofcomparisons) 
    paircompleteness = (truepositives)/(truepositives + falsenegatives)
  
         #print("Pair quality = ", str(pairquality))
#     print("Pair completeness = ", str(paircompleteness))
#     f1starmeasure = (2*pairquality*paircompleteness)/(pairquality+paircompleteness)
#     print("f1starmeasure = " + str(f1starmeasure))
    
    return (fracofcomparisons, fracofcomparisons2, f1measure, truepositives, falsepositives, falsenegatives, truenegatives, pairquality, paircompleteness)

In [ ]:
#create list with all model ID's
modelID_list = []
for i in range(1, len(new_data)+1):
    modelID_list.append(new_data[i]["modelID"])
    
# for i in range(0,len(modelID_list)):
#     modelID_list[i] = modelID_list[i].lower()    
modelID_list

In [ ]:
#create train and test set for each bootstrap iteration
bootstraps = 5
trainlist = []
testlist = []
for i in range(0, bootstraps):
    trainsample = rd.choices(range(0,len(modelID_list)),k=len(modelID_list))
    trainsample2 = list(dict.fromkeys(trainsample))
    trainset = sorted(trainsample2)
    full_data = list(range(0, len(modelID_list)))
    testset = list(set(full_data) - set(trainset))
    trainlist.append(list(trainset))
    testlist.append(list(testset))

In [ ]:
#create list with all titles
titles = []
for i in range(1, len(new_data.keys())+1):
     titles.append(new_data[i]["title"])

In [ ]:
%%time
list_f1star = []
list_f1 = []
list_compare = []
list_compare2 = []
list_pair_qual = []
list_pair_compl= []
list_truepos = []
list_falsepos = []
list_falseneg = []
list_trueneg = []
list_f1star_test = []
list_f1_test = []
list_compare_test = []
list_compare2_test = []
list_pair_qual_test = []
list_pair_compl_test = []
list_truepos_test = []
list_falsepos_test = []
list_falseneg_test = []
list_trueneg_test = []
#create random vectors for different parameter values
bandlength =  [1]
cos_threshold = [0.7, 0.8, 0.9]
beta =  [0.2, 0.3, 0.4] 
delta = [0.8, 0.9]
clustering_threshold = [0.2, 0.15, 0.17, 0.22]

for j in range(0, 5):
    param_list = [bandlength, cos_threshold, beta, delta, clustering_threshold]
    random_grid = rd.sample(list(itertools.product(*param_list)), 10)

    #random grid search with the different parameter vectors to obtain (close to) optimal parameter values based on the train set
    for i in range(0, len(random_grid)):
        print(i, "bandlength:" + str(random_grid[i][0]), "cosine threshold:" + str(random_grid[i][1]), "beta:" + str(random_grid[i][2]), "delta:" + str(random_grid[i][3]), "clustering threshold:"+str(random_grid[i][4]))
        results = model_LSH(trainlist[j], titles, modelID_list, random_grid[i][0], random_grid[i][1], random_grid[i][2], random_grid[i][3], random_grid[i][4])
        print(results) 
        fraccompare = results[0]
        fraccompare2 = results[1]
        f1 = results[2]
        truepos = results[3]
        falsepos = results[4]
        falseneg = results[5]
        trueneg = results[6]
        pairqual = results[7]
        paircompl = results[8]
        
        list_compare.append(fraccompare)
        list_compare2.append(fraccompare2)
#         list_f1star.append(f1star)
        list_f1.append(f1)
        list_truepos.append(truepos)
        list_falsepos.append(falsepos)
        list_falseneg.append(falseneg)
        list_trueneg.append(trueneg)
        list_pair_qual.append(pairqual)
        list_pair_compl.append(paircompl)
        
    max_value = max(list_f1)
    max_index = list_f1.index(max_value)
    best_parameters = random_grid[max_index]
    test_results = model_LSH(testlist[j], titles, modelID_list, best_parameters[0], best_parameters[1], best_parameters[2], best_parameters[3], best_parameters[4])
    print(test_results)
    fraccompare_test = test_results[0]
    fraccompare2_test = test_results[1]
#     f1star_test = test_results[2]
    f1_test = test_results[2]
    truepos_test = test_results[3]
    falsepos_test = test_results[4]
    falseneg_test = test_results[5]
    trueneg_test = test_results[6]
    pairqual_test = test_results[7]
    paircompl_test = test_results[8]
    
    f1star_test = (2*pairqual_test*paircompl_test)/(pairqual_test+paircompl_test)
    
    
    list_compare_test.append(fraccompare_test)
    list_compare2_test.append(fraccompare2_test)
    list_f1star_test.append(f1star_test)
    list_f1_test.append(f1_test)
    list_truepos_test.append(truepos_test)
    list_falsepos_test.append(falsepos_test)
    list_falseneg_test.append(falseneg_test)
    list_trueneg_test.append(trueneg_test)
    list_pair_qual_test.append(pairqual_test)
    list_pair_compl_test.append(paircompl_test)
    print(j)

In [ ]:
result_test_1_5 = model_LSH(testlist[4], titles, modelID_list, 1, 0.9, 0.3, 0.8, 0.15)

In [ ]:
total_result_1 = []
total_result_1.append(result_test_1[2])
total_result_1.append(result_test_1_2[2])
total_result_1.append(result_test_1_3[2])
total_result_1.append(result_test_1_4[2])
total_result_1.append(result_test_1_5[2])
total_result_1

In [ ]:
mean_f1_1

In [ ]:
mean_compare2_1

In [ ]:
total_result_1_compare2 = []
total_result_1_compare2.append(result_test_1[1])
total_result_1_compare2.append(result_test_1_2[1])
total_result_1_compare2.append(result_test_1_3[1])
total_result_1_compare2.append(result_test_1_4[1])
total_result_1_compare2.append(result_test_1_5[1])
total_result_1_compare2

In [ ]:
list_compare_test_14

In [ ]:
list_f1_graph = []
list_f1_graph.append(statistics.mean(list_f1_test_14))

In [ ]:
list_compare2f1_graph = []
list_compare2f1_graph.append(statistics.mean(list_compare2_test_14))

In [ ]:
statistics.mean(list_compare2_test_14)

In [ ]:
%%time
list_f1star = []
list_f1 = []
list_compare = []
list_compare2 = []
list_pair_qual = []
list_pair_compl= []
list_truepos = []
list_falsepos = []
list_falseneg = []
list_trueneg = []
list_f1star_test = []
list_f1_test = []
list_compare_test = []
list_compare2_test = []
list_pair_qual_test = []
list_pair_compl_test = []
list_truepos_test = []
list_falsepos_test = []
list_falseneg_test = []
list_trueneg_test = []
list_f1_test_bandlength = []
list_compare_test_bandlength = []
list_compare2_test_bandlength = []

#create random vectors for different parameter values
bandlength =  [7,5,4,3,1]
cos_threshold = [0.7, 0.8, 0.9]
beta =  [0.2, 0.3, 0.4] 
delta = [0.7, 0.8, 0.9]
clustering_threshold = [0.2, 0.15, 0.17, 0.22]

for z in bandlength:
    for j in range(0, 5):
        param_list = [cos_threshold, beta, delta, clustering_threshold]
        random_grid = rd.sample(list(itertools.product(*param_list)), 10)

        #random grid search with the different parameter vectors to obtain (close to) optimal parameter values based on the train set
        for i in range(0, len(random_grid)):
            print(i, "bandlength:" + str(z), "cosine threshold:" + str(random_grid[i][0]), "beta:" + str(random_grid[i][1]), "delta:" + str(random_grid[i][2]), "clustering threshold:"+str(random_grid[i][3]))
            results = model_LSH(trainlist[j], titles, modelID_list, z, random_grid[i][0], random_grid[i][1], random_grid[i][2], random_grid[i][3])
            print(results) 
            fraccompare = results[0]
            fraccompare2 = results[1]
            f1 = results[2]
            truepos = results[3]
            falsepos = results[4]
            falseneg = results[5]
            trueneg = results[6]
            pairqual = results[7]
            paircompl = results[8]

            list_compare.append(fraccompare)
            list_compare2.append(fraccompare2)
    #         list_f1star.append(f1star)
            list_f1.append(f1)
            list_truepos.append(truepos)
            list_falsepos.append(falsepos)
            list_falseneg.append(falseneg)
            list_trueneg.append(trueneg)
            list_pair_qual.append(pairqual)
            list_pair_compl.append(paircompl)

        max_value = max(list_f1)
        max_index = list_f1.index(max_value)
        best_parameters = random_grid[max_index]
        test_results = model_LSH(testlist[j], titles, modelID_list, z, best_parameters[0], best_parameters[1], best_parameters[2], best_parameters[3])
        print(test_results)
        fraccompare_test = test_results[0]
        fraccompare2_test = test_results[1]
    #     f1star_test = test_results[2]
        f1_test = test_results[2]
        truepos_test = test_results[3]
        falsepos_test = test_results[4]
        falseneg_test = test_results[5]
        trueneg_test = test_results[6]
        pairqual_test = test_results[7]
        paircompl_test = test_results[8]

#         f1star_test = (2*pairqual_test*paircompl_test)/(pairqual_test+paircompl_test)

        list_compare_test.append(fraccompare_test)
        list_compare2_test.append(fraccompare2_test)
#         list_f1star_test.append(f1star_test)
        list_f1_test.append(f1_test)
        list_truepos_test.append(truepos_test)
        list_falsepos_test.append(falsepos_test)
        list_falseneg_test.append(falseneg_test)
        list_trueneg_test.append(trueneg_test)
        list_pair_qual_test.append(pairqual_test)
        list_pair_compl_test.append(paircompl_test)
        print(j)
        
    list_f1_test_bandlength.append(list_f1_test)
    list_compare_test_bandlength.append(list_compare_test)
    list_compare2_test_bandlength.append(list_compare2_test)   
    print(z)

In [ ]:
list_f1_test_7 = []
list_f1_test_5 = []
list_f1_test_4 = []
list_f1_test_3 = []
for i in range (0,3):
    list_f1_test_7.append(list_f1_test_bandlength[0][i])
    list_f1_test_5.append(list_f1_test_bandlength[0][i+3])
    list_f1_test_4.append(list_f1_test_bandlength[0][i+6])
    list_f1_test_3.append(list_f1_test_bandlength[0][i+9])
list_f1_test_7

In [ ]:
list_f1_graph.append(statistics.mean(list_f1_test_7))
list_f1_graph.append(statistics.mean(list_f1_test_5))
list_f1_graph.append(statistics.mean(list_f1_test_4))
list_f1_graph.append(statistics.mean(list_f1_test_3))
list_f1_graph

In [ ]:
list_f1_test_bandlength

In [ ]:
list_compare2f1_test_7 = []
list_compare2f1_test_5 = []
list_compare2f1_test_4 = []
list_compare2f1_test_3 = []
for i in range (0,3):
    list_compare2f1_test_7.append(list_compare2_test_bandlength[0][i])
    list_compare2f1_test_5.append(list_compare2_test_bandlength[0][i+3])
    list_compare2f1_test_4.append(list_compare2_test_bandlength[0][i+6])
    list_compare2f1_test_3.append(list_compare2_test_bandlength[0][i+9])
list_compare2f1_test_7

In [ ]:
list_compare2f1_graph.append(statistics.mean(list_compare2f1_test_7))
list_compare2f1_graph.append(statistics.mean(list_compare2f1_test_5))
list_compare2f1_graph.append(statistics.mean(list_compare2f1_test_4))
list_compare2f1_graph.append(statistics.mean(list_compare2f1_test_3))

list_compare2f1_graph

In [ ]:
list_compare2f1_graph.append(mean_compare2_1)
list_f1_graph.append(mean_f1_1)

In [ ]:
plt.plot(list_compare2f1_graph, list_f1_graph, 'k')
plt.xlim(-0.01, 0.7)
plt.ylim(0, 0.5)
plt.xlabel("Fraction of Comparisons")
plt.ylabel("F1-measure")
plt.savefig('F1measure.png')

In [ ]:
list_f1star_test

In [ ]:
allresults = []
lsh_results = []
lsh_results_1 = []
list_bandlength = [14, 10, 7, 5, 4, 3, 1]
for j in list_bandlength:
    for i in range(0, bootstraps):
        lsh_results = model_LSH_only(testlist[i], titles, modelID_list, j)
        lsh_results_1.append(lsh_results)
        print(i)
    allresults.append(lsh_results_1)
    lsh_results_1 = []
    print(j)

In [ ]:
sum_compare = 0
sum_compare_list = []
sum_compare2 = 0
sum_compare2_list = []
sum_f1star = 0
sum_f1star_list = []
sum_quality = 0
sum_quality_list = []
sum_compl = 0
sum_compl_list = []
for j in range(0,7):
    for i in range(0,bootstraps):
        sum_compare = sum_compare + allresults[j][i][0]
        sum_compare2 = sum_compare2 + allresults[j][i][1]
        sum_f1star = sum_f1star + allresults[j][i][2]
        sum_quality = sum_quality + allresults[j][i][8]
        sum_compl = sum_compl + allresults[j][i][9]
    avg_compare = sum_compare / bootstraps
    avg_compare2 = sum_compare2 / bootstraps
    avg_f1star = sum_f1star / bootstraps
    avg_quality = sum_quality / bootstraps
    avg_compl = sum_compl / bootstraps
    sum_compare_list.append(avg_compare)
    sum_compare2_list.append(avg_compare2)
    sum_f1star_list.append(avg_f1star)
    sum_quality_list.append(avg_quality)
    sum_compl_list.append(avg_compl)
    sum_compare = 0
    sum_compare2 = 0
    sum_f1star = 0
    sum_quality = 0
    sum_compl = 0



In [ ]:
#plot f1* 
plt.plot(sum_compare2_list, sum_f1star_list, 'k')
plt.xlim(-0.01, 1)
plt.ylim(0, 0.35)
plt.xlabel("Fraction of Comparisons")
plt.ylabel("F1*-measure")
plt.savefig('F1starmeasure.png')

In [ ]:
#plot pair completeness
plt.plot(sum_compare2_list, sum_compl_list, 'k')
plt.xlim(-0.01, 1)
plt.ylim(0, 1.03)
plt.xlabel("Fraction of Comparisons")
plt.ylabel("Pair Completeness")
plt.savefig('Paircompleteness.png')

In [ ]:
#plot pair quality
plt.plot(sum_compare2_list, sum_quality_list, 'k')
plt.xlim(-0.001, 0.2)
plt.ylim(0, 0.3)
plt.xlabel("Fraction of Comparisons")
plt.ylabel("Pair Quality")
plt.savefig('Pairquality.png')

In [ ]:
brands = []
for i in trainset:
    if ('Brand' in new_data[i+1]['featuresMap']):
        brands.append(new_data[i+1]['featuresMap']['Brand'])
    elif ('Brand Name' in new_data[i+1]['featuresMap']):
        brands.append(new_data[i+1]['featuresMap']['Brand Name'])
    else:
        brands.append('No result')
for i in range(0,len(brands)):
    brands[i] = brands[i].lower() 
brandslist = [re.sub(r'lg electronics', 'lg', item) for item in brands]
brandslist2 = [re.sub(r'sceptre inc.', 'sceptre', item) for item in brandslist]
brands_final = [re.sub(r'jvc tv', 'jvc', item) for item in brandslist2]
brands_final

In [ ]:
def model_no_LSH(dataset, alltitles_list, modelID_list, cos_threshold, beta, delta, clustering_threshold):

    modelID = []
    for i in dataset:
        modelID.append(modelID_list[i])
    
    titles_dataset = []
    for i in dataset:
        titles_dataset.append(alltitles_list[i])
    
    tvlist = ["tv" + str(i) for i in dataset]
    
    lengthone = (len(modelID),len(modelID))
    sizeone = np.ones(lengthone)
    dfones = pd.DataFrame(sizeone)
    arronedata = dfones.to_numpy()
    newarronedata = np.triu(arronedata)
    dfnew5 = pd.DataFrame(newarronedata)

    list_per_tv = []
    for i in range(0, len(titles_dataset)):
        list_per_tv.append(titles_dataset[i].split())

    for i in range(0,len(list_per_tv)):
        for j in range(0, len(list_per_tv[i])):
            list_per_tv[i][j] = list_per_tv[i][j].lower()  

    import re

    pattern = re.compile("\d+")

    for i in range(0,len(list_per_tv)):
        for j in range(0, len(list_per_tv[i])-1):
            if (list_per_tv[i][j+1] == 'inch' or list_per_tv[i][j+1] == '”' or list_per_tv[i][j+1] == '"' or list_per_tv[i][j+1] == '-inch') and bool(pattern.match(list_per_tv[i][j])):
                new_word = list_per_tv[i][j] + 'inch'
                list_per_tv[i][j] = new_word
            elif (list_per_tv[i][j+1] == 'hz' or list_per_tv[i][j+1] == 'hertz' or list_per_tv[i][j+1] == '-hz') and bool(pattern.match(list_per_tv[i][j])):
                new_word2 = list_per_tv[i][j] + 'hz'
                list_per_tv[i][j] = new_word2


    filtered_per_tv = []            
    for i in range(0,len(list_per_tv)): 
        new_list3 = [s.replace("(", "") for s in list_per_tv[i]]
        new_list4 = [s.replace("[", "") for s in new_list3]
        r = re.compile("philips|samsung|supersonic|sharp|nec|toshiba|\
        hisense|sony|lg|sanyo|coby|panasonic|sansui|vizio|viewsonic|sunbritetv|haier|\
        proscan|jvc|pyle|lg electronics|sceptre|magnavox|mitsubishi|compaq|hannspree|\
        sceptre inc.|upstar|seiki|rca|craig|affinity|jvc tv|naxa|westinghouse|epson|hp|\
        elo|pansonic|hello kitty|sigmac|[a-z0-9]*[0-9]+[^0-9,(\s]*|[^0-9,(\s]*[0-9]+[a-z0-9]*")
        new_list5 = list(filter(r.match, new_list4))
        filtered_per_tv.append(new_list5)

    filtered_per_tv_final = []    
    for i in range(0,len(filtered_per_tv)):
        new_list6 = [re.sub(r'\"|\”|\'', 'inch', item) for item in filtered_per_tv[i]]
        new_list7 = [re.sub(r'-inch', 'inch', item) for item in new_list6]
        new_list8 = [re.sub(r'-hz|hertz', 'hz', item) for item in new_list7]
        new_list9 = [s.replace(".0", "") for s in new_list8]
        new_list10 = [s.replace(")", "") for s in new_list9]
    
        filtered_per_tv_final.append(new_list10) 

    filtered_final = []
    regex = re.compile(r'.{3,}inch')
    for i in range(0, len(filtered_per_tv_final)):
        filtered = [g for g in filtered_per_tv_final[i] if not regex.match(g)]
        filtered_final.append(filtered)

    shops = []
    for i in dataset:
         shops.append(new_data[i+1]["shop"])
            
    brands = []
    for i in dataset:
        if ('Brand' in new_data[i+1]['featuresMap']):
            brands.append(new_data[i+1]['featuresMap']['Brand'])
        elif ('Brand Name' in new_data[i+1]['featuresMap']):
            brands.append(new_data[i+1]['featuresMap']['Brand Name'])
        else:
            brands.append('no result')
            
    for i in range(0,len(brands)):
        brands[i] = brands[i].lower() 
    brandslist = [re.sub(r'lg electronics', 'lg', item) for item in brands]
    brandslist2 = [re.sub(r'sceptre inc.', 'sceptre', item) for item in brandslist]
    brands_final = [re.sub(r'jvc tv', 'jvc', item) for item in brandslist2]

    tvlengthnew5 = (len(tvlist),len(tvlist))
    Atvlengthnew5 = np.zeros(tvlengthnew5)
    dftvlistnew5 = pd.DataFrame(Atvlengthnew5, index=tvlist, columns=tvlist)

    for i in range(0, len(dataset)):
        for j in range(0, len(dataset)):
            if j > i:
                if dfnew5.iloc[i,j] == 1:
                    dftvlistnew5.iloc[i,j] = get_cosine_similarity(text_to_vector(titles_dataset[i]), text_to_vector(titles_dataset[j]))

    for i in range(0, len(dataset)):
        for j in range(0, len(dataset)):
            if j > i and shops[i] == shops[j]:
                dftvlistnew5.iloc[i,j] = -float('inf')
            if j > i and brands_final[i] != brands_final[j] and brands_final[i] != 'no result' and brands_final[j] != 'no result':
                dftvlistnew5.iloc[i,j] = -float('inf')

    numofcomp = dftvlistnew5

    numofcomp = numofcomp.replace(float('-inf'),0)
    numberofcomparisons = sum(numofcomp[numofcomp > 0].count())

    tvlength = (len(tvlist),len(tvlist))
    Atvlength = np.zeros(tvlength)
    dfsimlev = pd.DataFrame(Atvlength)

    tvlength1 = (len(tvlist),len(tvlist))
    Atvlength1 = np.zeros(tvlength1)
    dfsimlevMW = pd.DataFrame(Atvlength1)
    
    list1=[]
    regex = re.compile(r'[0-9]*')
    for i in range(0, len(filtered_final)):
        list2 = [re.sub(r'[0-9]*', '', item) for item in filtered_final[i]]
        list1.append(list2)

    list3=[]
    regex = re.compile(r'[^0-9]*')
    for i in range(0, len(filtered_final)):
        list4 = [re.sub(r'[^0-9]*', '', item) for item in filtered_final[i]]
        list3.append(list4)
    #TMWM
    sum_lev_MW = 0
    sum_lev = 0
    sum_length = 0
    sum_length_allpairs = 0
    for i in range(0, len(dftvlistnew5)):
        for j in range(0, len(dftvlistnew5)):
            if j > i and dfnew5.iloc[i,j] == 1 and dftvlistnew5.iloc[i,j] < cos_threshold and dftvlistnew5.iloc[i,j] >= 0:
                for z in range(0, len(filtered_final[i])):
                    for x in range(0, len(filtered_final[j])):
                        first_model = list1[i][z]
                        second_model = list1[j][x]
                        text_distance = textdistance.levenshtein.normalized_distance(first_model, second_model)
                        first_item = filtered_final[i][z]
                        second_item = filtered_final[j][x]
                        total_distance = textdistance.levenshtein.normalized_distance(first_item, second_item)
                        length_allpairs = len(filtered_final[i][z])+len(filtered_final[j][x])
                        sum_lev = sum_lev + (1-total_distance) * length_allpairs
                        sum_length_allpairs = sum_length_allpairs + length_allpairs
                        if text_distance < 0.1:
                            first_word = list3[i][z]
                            second_word = list3[j][x]
                            numeric_distance = textdistance.levenshtein.normalized_distance(first_word, second_word)
                            if numeric_distance > 0:
                                dftvlistnew5.iloc[i,j] = -float('inf')
                                length = len(filtered_final[i][z])+len(filtered_final[j][x])
                                sum_length = sum_length + length
                            else:
                                first_string = filtered_final[i][z]
                                second_string = filtered_final[j][x]
                                string_distance = textdistance.levenshtein.normalized_distance(first_string, second_string)
                                length = len(filtered_final[i][z])+len(filtered_final[j][x])
                                sum_lev_MW = sum_lev_MW + (1-string_distance) * length
                                sum_length = sum_length + length
            if dftvlistnew5.iloc[i,j] != 0 and sum_length != 0:
                avg_lev = sum_lev / sum_length_allpairs
                avg_lev_MW = sum_lev_MW / sum_length
                dfsimlev.iloc[i,j] = beta * dftvlistnew5.iloc[i,j] + (1-beta) * avg_lev
                dfsimlevMW.iloc[i,j] = delta * avg_lev_MW + (1-delta) * dfsimlev.iloc[i,j]
                sum_lev_MW = 0
                sum_length = 0
                sum_lev = 0
                sum_length_allpairs = 0

    for i in range(0, len(dfsimlevMW)):
        for j in range(0, len(dfsimlevMW)):
            if j > i and shops[i] == shops[j]:
                dfsimlevMW.iloc[i,j] = -float('inf')
            if j > i and brands_final[i] != brands_final[j] and brands_final[i] != 'no result' and brands_final[j] != 'no result':
                dfsimlevMW.iloc[i,j] = -float('inf')



    finalm = (len(dfsimlevMW),len(dfsimlevMW))
    finalA = np.ones(finalm)
    onematrix = pd.DataFrame(finalA)
    dfsimlevMWone = onematrix.sub(dfsimlevMW)
    finaldissimilaritymatrix = dfsimlevMWone.replace(1, 1000)
    finaldissimilaritymatrix2 = finaldissimilaritymatrix.replace(float('inf'), 1000)
    finaldissimilaritymatrix2numpy = finaldissimilaritymatrix2.to_numpy()
    i_lower = np.tril_indices(len(tvlist), -1)
    finaldissimilaritymatrix2numpy[i_lower] = finaldissimilaritymatrix2numpy.T[i_lower]
    dffinaldissim = pd.DataFrame(finaldissimilaritymatrix2numpy)

    clustering = AgglomerativeClustering(n_clusters = None, affinity='precomputed', linkage='complete', distance_threshold=clustering_threshold, compute_full_tree= True).fit(dffinaldissim)
    labelsclust = clustering.labels_

    originalm = (len(modelID),len(modelID))
    originalA = np.zeros(originalm)
    originalcorrect = pd.DataFrame(originalA, index=modelID, columns=modelID)
    for i in range(0, len(modelID)):
        for j in range(0, len(modelID)):
            if modelID[i] == modelID[j]:
                originalcorrect.iloc[i,j] = 1
    originalcorrect.values[[np.arange(originalcorrect.shape[0])]*2] = 0

    clusterdm = (len(modelID),len(modelID))
    clusteredA = np.zeros(clusterdm)
    clusteredmat = pd.DataFrame(clusteredA, index=modelID, columns=modelID)
    check = pd.DataFrame(list(zip(labelsclust, modelID)))
    for i in range(0, len(modelID)):
        for j in range(0, len(modelID)):
            if check[0][i] == check[0][j]:
                clusteredmat.iloc[i,j] = 1
    clusteredmat.values[[np.arange(clusteredmat.shape[0])]*2] = 0

    falsenegatives = 0
    falsepositives = 0
    truenegatives = 0
    truepositives = 0

    for i in range(0, len(originalcorrect)):
        for j in range(0, len(originalcorrect)):
            if j > i:
                if  clusteredmat.iloc[i,j] == 0 and originalcorrect.iloc[i,j] == 0:
                    truenegatives = truenegatives + 1
                elif  clusteredmat.iloc[i,j] == 0 and originalcorrect.iloc[i,j] == 1:
                    falsenegatives = falsenegatives + 1
                elif  clusteredmat.iloc[i,j] == 1 and originalcorrect.iloc[i,j] == 0:
                    falsepositives = falsepositives + 1
                elif  clusteredmat.iloc[i,j] == 1 and originalcorrect.iloc[i,j] == 1:
                    truepositives = truepositives + 1

#     print("truepositives = "+ str(truepositives))
#     print("truenegatives = " + str(truenegatives))
#     print("falsepositives = " + str(falsepositives))
#     print("falsenegatives = "+ str(falsenegatives))
    f1measure = (truepositives/(truepositives+0.5*(falsenegatives+falsepositives)))
#     print("f1measure = " + str(f1measure))

    pairquality = (truepositives)/(numberofcomparisons) 
    paircompleteness = (truepositives)/(truepositives + falsenegatives)
#     print("Pair quality = ", str(pairquality))
#     print("Pair completeness = ", str(paircompleteness))
    f1starmeasure = (2*pairquality*paircompleteness)/(pairquality+paircompleteness)
#     print("f1starmeasure = " + str(f1starmeasure))
    
    return (fracofcomparisons, fracofcomparisons2, f1starmeasure, f1measure, truepositives, falsepositives, falsenegatives, truenegatives, pairquality, paircompleteness) 